In [4]:
import dicom2nifti

dicom2nifti.convert_directory(r"YOUR_DATA_PATH\t2_de3d_we_cor_iso_0003", r"YOUR_DATA_PATH", compression=True, reorient=True)
dicom2nifti.convert_directory(r"YOUR_DATA_PATH\t2_space_cor_iso_myelo_ZOOMit_0004", r"YOUR_DATA_PATH", compression=True, reorient=True)




In [15]:
dicom2nifti.convert_directory(r"YOUR_DATA_PATH\t2_de3d_we_sag_p2_iso_0013", r"YOUR_DATA_PATH", compression=True, reorient=True)


In [16]:
import SimpleITK as sitk

# Load the DESS and T2 images
dess = sitk.ReadImage(r"YOUR_DATA_PATH\13_t2_de3d_we_sag_p2_iso.nii.gz")
t2 = sitk.ReadImage(r"YOUR_DATA_PATH\4_t2_space_cor_iso_myelo_zoomit.nii.gz")

# Define the registration method
registration_method = sitk.ImageRegistrationMethod()

# Set the registration method parameters
registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
registration_method.SetMetricSamplingPercentage(0.01)
registration_method.SetInterpolator(sitk.sitkLinear)

# Define the registration transform
initial_transform = sitk.CenteredTransformInitializer(dess, t2, sitk.AffineTransform(dess.GetDimension()))
registration_method.SetInitialTransform(initial_transform)
registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=100, estimateLearningRate=registration_method.EachIteration)
registration_method.SetOptimizerScalesFromPhysicalShift()


dess_float = sitk.Cast(dess, sitk.sitkFloat32)
t2_float= sitk.Cast(t2, sitk.sitkFloat32)
# Run the registration
final_transform = registration_method.Execute(dess_float, t2_float)

# Apply the transform to the DESS image
registered_dess = sitk.Resample(dess, t2, final_transform, sitk.sitkLinear, 0.0, dess.GetPixelID())

# Save the registered DESS image
sitk.WriteImage(registered_dess, 'registered_dess_coronal.nii.gz')


In [13]:
dess

<SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'std::vector< itk::simple::Image >::value_type *' at 0x000001D011E673C0> >

In [10]:
t2

<SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'std::vector< itk::simple::Image >::value_type *' at 0x000001D018420D80> >